In [14]:
!pip install psycopg2-binary gspread gspread-dataframe pandas oauth2client

In [15]:
# Celda de Imports (debería ser tu primera o segunda celda de código)
import psycopg2
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
import json
import os
from google.colab import auth
from google.auth import default

In [16]:
import gspread
import json
import os

# Autenticación inteligente: usa la cuenta del robot en GitHub o tu cuenta en Colab
if 'GCP_SA_KEY' in os.environ:
    print("Autenticando con Cuenta de Servicio desde GitHub Secrets...")
    keyfile_dict = json.loads(os.environ['GCP_SA_KEY'])
    gc = gspread.service_account_from_dict(keyfile_dict)
else:
    print("Autenticando con usuario de Colab...")
    from google.colab import auth
    auth.authenticate_user()
    from google.auth import default
    creds, _ = default()
    gc = gspread.authorize(creds)

Autenticando con usuario de Colab...


In [17]:
import os

# Esta sección busca la contraseña.
# Si está en GitHub, la lee de los "secrets".
# Si está en Colab, la lee del gestor de secretos de Colab.
# Si no, la pones a mano (menos seguro).

if 'DB_PASSWORD' in os.environ:
    db_password = os.environ['DB_PASSWORD']
    print("Contraseña leída desde GitHub Secrets.")
else:
    try:
        from google.colab import userdata
        db_password = userdata.get('DB_PASSWORD')
        print("Contraseña leída desde Colab Secrets.")
    except:
        db_password = "TU_CONTRASEÑA_AQUÍ" # <-- Mantén esto para pruebas locales
        print("Usando contraseña local.")

Contraseña leída desde Colab Secrets.


In [18]:
# --------------- CONFIGURACIÓN (ajusta estas 4 líneas) ---------------
db_host = 'ec2-52-206-224-222.compute-1.amazonaws.com'
db_name = 'd77mgnf951v3g0'
db_user = 'reports'
sheet_name = 'Prueba automatización COLAB' # <-- El nombre exacto de tu Google Sheet
sql_query = """
    SELECT id, name, origin, phone
    FROM stores
    LIMIT 10
"""
# ---------------------------------------------------------------------

print("Iniciando proceso...")

try:
    # 1. Conectar a PostgreSQL
    print("Conectando a la base de datos...")
    conn = psycopg2.connect(
        host=db_host,
        dbname=db_name,
        user=db_user,
        password=db_password,
        port=5432
    )

    # 2. Ejecutar la query y cargarla en un DataFrame de Pandas
    print("Ejecutando consulta...")
    df = pd.read_sql_query(sql_query, conn)
    print(f"Se encontraron {len(df)} filas.")

    # 3. Conectar a Google Sheets y escribir los datos
    print(f"Abriendo Google Sheet: '{sheet_name}'...")
    spreadsheet = gc.open(sheet_name)
    worksheet = spreadsheet.get_worksheet(0) # Obtiene la primera hoja (pestaña)

    print("Limpiando la hoja y escribiendo los nuevos datos...")
    worksheet.clear()
    set_with_dataframe(worksheet, df)

    print("\n¡Proceso completado con éxito! Tu Google Sheet ha sido actualizado.")

except Exception as e:
    print(f"\nERROR: Ha ocurrido un problema.")
    print(e)

finally:
    # 4. Asegurarse de que la conexión siempre se cierre
    if 'conn' in locals() and conn is not None:
        conn.close()
        print("Conexión a la base de datos cerrada.")

Iniciando proceso...
Conectando a la base de datos...
Ejecutando consulta...
Se encontraron 10 filas.
Abriendo Google Sheet: 'Prueba automatización COLAB'...


/tmp/ipython-input-1130364318.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


Limpiando la hoja y escribiendo los nuevos datos...

¡Proceso completado con éxito! Tu Google Sheet ha sido actualizado.
Conexión a la base de datos cerrada.
